# Entrenamiento de los modelos

En este notebook se detalla el proceso de entrenamiento de los modelos utilizados en el proyecto.


## Puesta a punto

En la celda siguiente se cargan las librerías necesarias para garantizar el correcto funcionamiento del notebook.


In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from src.utils import best_model_per_grid, get_tuned_model
import pickle
import os

## Importación de los datos

Los datos utilizados en esta etapa fueron previamente almacenados en formato tabular dentro de un archivo CSV. A continuación, se procede a su carga para iniciar el proceso de entrenamiento de los modelos.


In [2]:
os.chdir('../')

In [3]:
X = pd.read_csv('data/processed/dataset.csv')

In [4]:
# Variables elegidas para el entrenamiento
X.columns

Index(['Edad', 'Sexo', 'Café', 'Tabaco', 'Alcohol', 'APPHTA', 'APPDM',
       'APPEPOC', 'APPIRC', 'APPEnfValvular', 'APPIMA', 'APPAngina', 'APPACV',
       'No.LesionesCoronarias', 'LesionACD', 'Lesion TCI', 'LesionADA',
       'LesionACircunfleja', 'BCPIAoPrep', 'BCPIAoTrans', 'BCPIAoPost',
       'Dobutamina', 'Dopamina', 'Norepinefrina', 'Epinefrina',
       'Nitroglicerina', 'Hipoglucemia', 'Hiperglucemia', 'Hiponatremia',
       'Hipernatremia ', 'Hipopotasemia', 'Hiperpotasemia',
       'Acidosis metabólica', 'Alcalosis metabólica', 'Acidosis respiratoria',
       'Alcalosis respiratoria', 'DisfRenalPosop', 'DisfNeuroPosop',
       'DisfHepatPosop', 'Estadia', 'Egreso', 'Evoluciòn', 'FEVIPreop',
       'TamañoVI', 'AltContractVI', 'AMI', 'AMI+VSI', 'DAMI', 'DAM + VS',
       'PuentesAR', 'PuentesAGep', 'PuentesVen', 'RevascIncompleta', 'CEC',
       'DuracionCEC', 'Uso vasoactivos pst', 'AltMITransop', 'IMAPeriop',
       'Vasoplejia post', 'Hipoxemia post ', 'PaO2/FiO2 post',
 

In [5]:
# Separar las variables predictoras (X) de la objetivo (y)
X, y = X.drop('SBGC', axis=1), X['SBGC']

## Definición y entrenamiento

Se decidió entrenar dos modelos de clasificación supervisada: un **DecisionTreeClassifier** y un **RandomForestClassifier**, con el objetivo de comparar su desempeño y seleccionar el que ofrezca mejores resultados en la predicción del Síndrome de Bajo Gasto Cardíaco.

En una primera etapa, ambos modelos se definirán estableciendo únicamente el parámetro `random_state` para garantizar la reproducibilidad de los resultados. La selección de los hiperparámetros óptimos se realizará posteriormente mediante *GridSearchCV*, una estrategia viable dado el tamaño reducido del conjunto de datos.


In [6]:
rf_model = RandomForestClassifier(random_state=42)

In [7]:
# Parámetros utilizados por el GridSearchCV para RandomForestClassifier
rf_params = {
    'n_estimators': [200, 300, 350],
    'max_depth': [6, 10, 12],
    'min_samples_leaf': [2, 4, 6],
    'min_samples_split': [2, 4, 10],
    'criterion': ['gini', 'entropy']
}

In [8]:
#  RandomForestClassifier con los parámetros que maximizan su precisión
rf_model = best_model_per_grid(rf_model, rf_params, X, y)

In [9]:
dt_model = DecisionTreeClassifier(random_state=42)

In [10]:
# Parámetros utilizados por el GridSearchCV para DecisionTreeClassifier
dt_params = {
    'max_depth': [5, 6, 7, 8, 9, 10, 11, 12],
    'max_features': ['sqrt', 'log2', 0.2, 0.5, 0.8],
    'min_samples_leaf': [4, 5, 6, 7, 8, 9, 10, 11, 12],
    'min_samples_split': [4, 5, 6, 7, 8, 9, 10, 11, 12],
    'criterion': ['gini', 'entropy']
}

In [18]:
# DecisionTreeClassifier con los parámetros que maximizan su precisión
dt_model = best_model_per_grid(dt_model, dt_params, X, y)

## Refinamiento

Se aplicó un procedimiento de selección automática de características, conservando aquellas cuyo peso era superior al valor medio, con el objetivo de reducir la dimensionalidad y mantener únicamente las variables más relevantes para la predicción.

In [12]:
# RandomForestClassifier solo con las variables más relevantes
rf_tuned_model, rf_features = get_tuned_model(rf_model, X, y)

# Variables escogidas
rf_features

Index(['Dobutamina', 'Norepinefrina', 'Acidosis metabólica', 'FEVIPreop',
       'TamañoVI', 'DuracionCEC', 'PaO2/FiO2 post', 'Lactato post',
       'FEVI post', 'SvO2'],
      dtype='object')

In [13]:
# Media de la precisión obtenida por el RandomForestClassifier en cada partición de la validación cruzada
rf_accuracy = cross_val_score(rf_tuned_model, X[rf_features], y, cv=5, scoring='accuracy').mean()

rf_accuracy

0.9826086956521738

In [14]:
# DecisionTreeClassifier solo con las variables más relevantes
dt_tuned_model, dt_features = get_tuned_model(dt_model, X, y)

dt_features

Index(['CEC', 'FEVI post'], dtype='object')

In [15]:
# Media de la precisión obtenida por el DecisionTreeClassifier en cada partición de la validación cruzada
dt_accuracy = cross_val_score(dt_tuned_model, X[dt_features], y, cv=5, scoring='accuracy').mean()

dt_accuracy

0.9742753623188406

A pesar de que el DecisionTreeClassifier presentó una precisión inicial mayor, fue luego superado por el RandomForestClassifier refinado.

In [16]:
# Almacenar el RandomForestClassifier refinado
with open('models/rf_model.pkl', 'wb') as file:
    pickle.dump(rf_tuned_model, file)

In [17]:
# Almacenar el DecisionTreeClassifier refinado
with open('models/dt_model.pkl', 'wb') as file:
    pickle.dump(dt_tuned_model, file)